# Projet Scapy

## 1. Initiation à Scapy

On importe le module scapy et on vérifie son fonctionnement a l'aide de quelque **print()**.

In [3]:
from scapy.all import *
print(f"La version de Scapy est {conf.version}.")
print(f"\nL'interface par défaut utilisée pour l'émission et la réception des paquets est {conf.iface}.")
print(f"\nLa table de routage utilisée est : \n {conf.route}.")
print('\nLa passerelle par défaut est :', conf.route.route("0.0.0.0")[2])

La version de Scapy est 2.6.1.

L'interface par défaut utilisée pour l'émission et la réception des paquets est \Device\NPF_{3D7B1027-8CC4-4D67-88E6-0F92F60DFA95}.

La table de routage utilisée est : 
 Network          Netmask          Gateway      Iface                                           Output IP        Metric
0.0.0.0          0.0.0.0          192.168.1.1  Realtek 8822CE Wireless LAN 802.11ac PCI-E NIC  192.168.1.13     35    
127.0.0.0        255.0.0.0        0.0.0.0      Software Loopback Interface 1                   127.0.0.1        331   
127.0.0.1        255.255.255.255  0.0.0.0      Software Loopback Interface 1                   127.0.0.1        331   
127.255.255.255  255.255.255.255  0.0.0.0      Software Loopback Interface 1                   127.0.0.1        331   
192.168.1.0      255.255.255.0    0.0.0.0      Realtek 8822CE Wireless LAN 802.11ac PCI-E NIC  192.168.1.13     291   
192.168.1.13     255.255.255.255  0.0.0.0      Realtek 8822CE Wireless LAN 802.11ac 

> Le module est correctement installer, on importe alors le fichier *.pcap* qui correspond au trames d'un **ping vers google** (8.8.8.8).

In [12]:
trames = rdpcap("Ping_Google.pcapng")
print("Le fichier contient les trames suivante :")
trames.summary()

Le fichier contient les trames suivante :
Ether / IP / ICMP 192.168.1.48 > 8.8.8.8 echo-request 0 / Raw
Ether / IP / ICMP 8.8.8.8 > 192.168.1.48 echo-reply 0 / Raw
Ether / IP / ICMP 192.168.1.48 > 8.8.8.8 echo-request 0 / Raw
Ether / IP / ICMP 8.8.8.8 > 192.168.1.48 echo-reply 0 / Raw


Avec quelque description (fonctionne comme un dictionaire python) on peut retrouver la data émissent par notre paquet et aussi le décodé

In [14]:
#On recherche ici dans la trames 3 (écrit 2 car fonctionnement des dictionnaire python) la 3 ligne de la data que l'on décode en "UTF-8"
trames[2][Raw].load.split(sep=None)[2].decode('UTF8')

'!"#$%&\'()*+,-./01234567'

## Emission de trames

Le code ci-dessous permet d'emettre un paquet *ICMP* vers ici la destination 192.168.1.1.

In [13]:
ping=IP(dst='192.168.1.1')/ICMP()/"On peut ajouter ici des données à émettre avec le ping"
send(ping)


Sent 1 packets.


 * En regardant sur Wireshark on voit bien que notre packet a été emis :

<img src="etape_1.png">

## Création d'un programme "Sniffer"

Le but d'un programme Siffer est de récupérée les trames émissent ou reçu par la machine (à la manière de Wireshark)

 * Ici, Il récupère les emission et réception de paquet icmp comme ici dans un échange entre la machine et la passerelle (192.168.1.1)

In [3]:
from scapy.all import *
ICMP_types={ 0 : 'Echo-Reply', 3 : 'Destination Unreachable', 8 : 'Echo'}
def print_icmp (packet) :
    type=packet[ICMP].type
    ips=packet[IP].src
    ipd=packet[IP].dst
    if ips==iface_ip :
        print(f"Emission d'un paquet ICMP {ICMP_types[type]} vers {ipd}")
    else :
        print(f"Réception d'un paquet ICMP {ICMP_types[type]} en provenance de {ips}")
iface_ip=get_if_addr(conf.iface)
sniff(filter="icmp", prn=print_icmp, store=0, iface='Wi-Fi',count=4)

Emission d'un paquet ICMP Echo vers 8.8.8.8
Emission d'un paquet ICMP Echo vers 8.8.8.8
Emission d'un paquet ICMP Echo vers 8.8.8.8
Emission d'un paquet ICMP Echo vers 8.1.8.8


<Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>